In [7]:
import os
os.chdir('../')
%pwd

'd:\\ML Block\\Projects\\ETE AI Projects\\kidney-disease-classification-dl-project'

In [30]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/kvamsi7/kidney-disease-classification-dl-project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "kvamsi7"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "d6b1c8cf96f5b7cc6f735f6e0c8d127e5ad9c82d"


# export MLFLOW_TRACKING_URI = "https://dagshub.com/kvamsi7/kidney-disease-classification-dl-project.mlflow"

# export MLFLOW_TRACKING_USERNAME = "kvamsi7"

# export MLFLOW_TRACKING_PASSWORD = "d6b1c8cf96f5b7cc6f735f6e0c8d127e5ad9c82d"

In [1]:
import tensorflow as tf

In [14]:
# model = tf.keras.models.load_model(f"/artifacts/training/model.h5")
model_path = r"artifacts\training\model.h5"
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
else:
    print(f"File not found: {model_path}")

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [ ]:
import dagshub
dagshub.init(repo_owner='kvamsi7', repo_name='kidney-disease-classification-dl-project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [16]:
from src.kidney_cnn_classifier.constants import *
from src.kidney_cnn_classifier.utils.common import read_yaml,create_directories,save_json

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = r"artifacts\training\model.h5",
            training_data = r"artifacts\data_ingestion\kidney_ct_scan_image_normal_tumor",
            mlflow_uri = "https://dagshub.com/kvamsi7/kidney-disease-classification-dl-project.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [28]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub

ModuleNotFoundError: No module named 'dagshub'

In [25]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}
        save_json(path = Path("scores.json"), data = scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_experiment("kidney_disease_classification_tracking")
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")

In [26]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-22 13:30:37,362: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-22 13:30:37,364: INFO: common: yaml file: params.yaml loaded successfully]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 123s 887ms/step - loss: 7.3771 - accuracy: 0.6978
[2024-11-22 13:32:40,340: INFO: common: json file saved at: scores.json]


2024/11/22 13:32:40 INFO mlflow.tracking.fluent: Experiment with name 'kidney_disease_classification_tracking' does not exist. Creating a new experiment.
2024/11/22 13:32:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\vamsi7\AppData\Local\Temp\tmpkn1matub\model\data\model\assets
[2024-11-22 13:32:41,691: INFO: builder_impl: Assets written to: C:\Users\vamsi7\AppData\Local\Temp\tmpkn1matub\model\data\model\assets]


2024/11/22 13:32:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
